In [1]:
import sys
sys.path.append("../src/")
from plugin_write_and_run import write_and_run

In [2]:
%%write_and_run ../src/main.py
#!/usr/bin/python

import sys
sys.path.append("../src/")
import gym
import random
import time
from config import *
from dddqn_agent import *
from dueling_dqn_network import *
from pong_wrapper import *
from process_image import *
from replay_buffer import *
from utilities import *

In [3]:
%%write_and_run -a ../src/main.py

pong_wrapper = PongWrapper(ENV_NAME, NO_OP_STEPS)
print("The environment has the following {} actions: {}".format(pong_wrapper.env.action_space.n, pong_wrapper.env.unwrapped.get_action_meanings()))

The environment has the following 6 actions: ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']


In [4]:
%%write_and_run -a ../src/main.py

MAIN_DQN = build_q_network(pong_wrapper.env.action_space.n, LEARNING_RATE, input_shape=INPUT_SHAPE)
TARGET_DQN = build_q_network(pong_wrapper.env.action_space.n, input_shape=INPUT_SHAPE)

replay_buffer = ReplayBuffer(size=MEMORY_SIZE, input_shape=INPUT_SHAPE)
dddqn_agent = DDDQN_AGENT(MAIN_DQN, TARGET_DQN, replay_buffer, pong_wrapper.env.action_space.n, 
                    input_shape=INPUT_SHAPE, batch_size=BATCH_SIZE, 
                   replay_buffer_start_size=REPLAY_MEMORY_START_SIZE,
                   max_frames=MAX_FRAMES)

In [5]:
%%write_and_run -a ../src/main.py

if PATH_LOAD_MODEL is not None:
    start_time = time.time()
    print('Loading model and info from the folder ', LOAD_FROM)
    info = dddqn_agent.load(LOAD_FROM, LOAD_REPLAY_BUFFER)

    # Apply information loaded from meta
    frame_number = info['frame_number']
    rewards = info['rewards']
    loss_list = info['loss_list']

    print(f'Loaded in {time.time() - start_time:.1f} seconds')
else:
    frame_number = 0
    rewards = []
    loss_list = []

In [7]:
while frame_number < MAX_FRAMES:
    epoch_frame = 0
    while epoch_frame < EVAL_FREQUENCY:
        start_time = time.time()
        pong_wrapper.reset()
        episode_reward_sum = 0
        for _ in range(MAX_EPISODE_LENGTH):
            action = dddqn_agent.get_action(frame_number, pong_wrapper.state)
            processed_frame, reward, terminal = pong_wrapper.step(action)
            frame_number += 1
            epoch_frame += 1
            episode_reward_sum += reward

            # Add experience to replay memory
            dddqn_agent.add_experience(action=action,
                                 frame=processed_frame[:, :, 0], # shape 84x84, remove last dimension
                                 reward=reward, clip_reward=CLIP_REWARD,
                                 terminal=terminal)

            # Update agent
            if frame_number % UPDATE_FREQ == 0 and dddqn_agent.replay_buffer.count > REPLAY_MEMORY_START_SIZE:
                loss, _ = dddqn_agent.learn(BATCH_SIZE, gamma=DISCOUNT_FACTOR)
                loss_list.append(loss)

            # Update target network
            if frame_number % NETW_UPDATE_FREQ == 0 and frame_number > REPLAY_MEMORY_START_SIZE:
                agent.update_target_network()

            # Break the loop when the game is over
            if terminal:
                terminal = False
                break

        rewards.append(episode_reward_sum)

        # Output the progress every 10 games
        if len(rewards) % 10 == 0:
            # Print results
            print(f'Game number: {str(len(rewards)).zfill(6)}  Frame number: {str(frame_number).zfill(8)}  Average reward: {np.mean(rewards[-10:]):0.1f}  Time taken: {(time.time() - start_time):.1f}s')

    # Evaluation
    terminal = True
    eval_rewards = []
    evaluate_frame_number = 0

    for _ in range(EVAL_LENGTH):
        if terminal:
            game_wrapper.reset(evaluation=True)
            life_lost = True
            episode_reward_sum = 0
            terminal = False

        action = dddqn_agent.get_action(frame_number, pong_wrapper.state, evaluation=True)

        # Step action
        _, reward, terminal = pong_wrapper.step(action)
        evaluate_frame_number += 1
        episode_reward_sum += reward

        # On game-over
        if terminal:
            eval_rewards.append(episode_reward_sum)

    if len(eval_rewards) > 0:
        final_score = np.mean(eval_rewards)
    else:
        # In case the first game is longer than EVAL_LENGHT
        final_score = episode_reward_sum
    # Print score
    print('Evaluation score:', final_score)
    
    # Save model
    if len(rewards) > 300 and SAVE_PATH is not None:
        dddqn_agent.save(f'{SAVE_PATH}/save-{str(frame_number).zfill(8)}', frame_number=frame_number, rewards=rewards, loss_list=loss_list)

Game number: 000010  Frame number: 00009647  Average reward: -20.1  Time taken: 0.7s
Game number: 000020  Frame number: 00018625  Average reward: -20.4  Time taken: 0.8s
Game number: 000030  Frame number: 00028277  Average reward: -20.2  Time taken: 1.1s
Game number: 000040  Frame number: 00037145  Average reward: -20.5  Time taken: 0.7s
Game number: 000050  Frame number: 00046787  Average reward: -20.0  Time taken: 0.9s
Game number: 000060  Frame number: 00055523  Average reward: -20.5  Time taken: 85.9s


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-68b0696187d4>", line 22, in <module>
    loss, _ = dddqn_agent.learn(BATCH_SIZE, gamma=DISCOUNT_FACTOR)
  File "../src/dddqn_agent.py", line 115, in learn
    model_gradients = tape.gradient(loss, self.DQN.trainable_variables)
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py", line 1080, in gradient
    flat_grad = imperative_grad.imperative_grad(
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/imperative_grad.py", line 71, in imperative_grad
    return pywrap_tfe.TFE_Py_TapeGradient(
  File "/Users/anton/.conda/envs/reinforcement_learning/lib/python3.8/site-packages/tensorflow/python/eager/backprop.py", l

TypeError: object of type 'NoneType' has no len()